# Goals
Models are either capped at `1024` or `512` tokens. To summarize our notes, let's split each note into appropriate sections by reading in the headers. Each section will be summarized separately,and those strings will then be concatenated together

In [4]:
import markdown
from transformers import pipeline, AutoTokenizer
from functions import summarize, summarize_string

In [5]:
MAX_HEADER_DEPTH = 6

In [6]:
long_note = "/Users/joachimpfefferkorn/repos/daily_note_organizer/test_media/2023-10-17.md"

In [7]:

def clean_md(md_content: str):
    #TODO
    # Remove hyperlinks
    # Disregard edge cases, anything encapsulated in a code block
    # use a look up table of strings to replace instead of this shadowed variable
    clean_note = md_content
    clean_note = clean_note.replace('- [x]', 'Completed:').replace('- [ ]','To Do:')
    clean_note = clean_note.replace('[[', '').replace(']]','')
    clean_note = clean_note.replace('![[', 'Image file:')
    return clean_note

def prepare_note(md_path):
    with open(md_path, 'r') as note:
        md_content = note.read()
        cleaned_md = clean_md(md_content)
        return cleaned_md

In [111]:
prepared_note = prepare_note(str(long_note))
# print(type(prepared_note))
#print(prepared_note)

In [77]:
bad_python = """"Automate the Boring Stuff with Python?" by "Al Sweigart" sounds like a book in which you will learn
to "automate" something?" You do not! "Automate the Boring Stuff with Python" leaves me floundering with
"incomplete" examples of code! Please, provide complete examples. Do not leave it to me to complete your
partial examples. I am trying to LEARN the language now, not wander on the path of the unknown. I am new to
Monty Python's Language! Guido van Rossum didn't create anything. He "developed" something!
What he developed is incomplete, confusing and not well thought out! The language is dependent on
"Indentation!" This is a poor choice! Comments are limited to "#" and do not encompass multiple lines
of code like the "C" language family's comment structure of (/* ... */!) This is another of "Al Sweigart's"
disappointing books! Python requires a paradigm shift that I am trying to make after 30 years of IBM Mainframe 360 familiy to today's
"System Z" of coding IBM Assembler, PL/1, REXX, and various "Print Scripting Languages," debugging, advising, co-operating with others
in IT, and writing internal help guides for my co-workers! My last job I was designated a "Principal Engineer," 1 of 2,000 IT
employees. There were 20 of us at the time. I DO know something about coding, debugging and sharing my knowledge with others.
I am not bragging! This is just a fact! I am disappointed with "Automate the Boring Stuff with Python" and do hope that "Al Sweigart"
will write, or edit his books, to be complete for us "beginners" trying to slog along the path of learnig a "Not" supposedly "Easy To
Learn Programming Language" that many major development companies use! I am not dumb. My IQ "was" quantified at 143 when I was a
teenager. I did attended the United States Military Academy (USMA) at West Point, NY. for two years. I have an "Honorable Discharge"
and left due to a torn ligament in the knee of my left leg. Year end PT did not go well! I ran for part of the course, and walked the
remainder. My "time" at the end was below the required.
West Point was hard to get into, and harder to stay. We carried 23 hour's of "Not Easy" course work per Semester!
This included my choice of the"Russian Language!," a well developed language, but difficult to learn! Looking back,
I should have picked "Spanish" considering where my family chose to live in my waning years! I left the University
I was attending as a Freshman, started over as a Plebe (4th Classman, Freshman,) at USMA, WP, NY., stayed 2 years,
and returned to my original University and graduated with a BS in Math, Comp Sci., Life Sci., or, as the Degree was
called, "General Science." Please, "Mr. Al Sweigart" write books that are "Complete" for those beginneres, trying
to learn a new programming language!" I do Thank You "Mr. Al Sweigart" for you efforts! Please, write "Complete"
books for the newbies sinking in the many programming languages coming and going today!"""

print(summarize_string(bad_python, summarizer, tokenizer))
print(type(bad_python))

[{'summary_text': '"Automate the Boring Stuff with Python" by "Al Sweigart" is a book in which you will learn to code. I am new to Monty Python\'s Language. I know a lot about coding, debugging and sharing my knowledge with others. My IQ was 143 when I was a teenager. I attended the United States Military Academy at West Point, NY. I worked as an engineer for 30 years coding IBM Assembler, PL/1, REXX, and various "Print Scripting Languages," debugging, advising, co-operating with others, and writing internal help guides for my co-workers. I was designated a "Principal Engineer," 1 of 2,000 IT employees, but left due to injury. '}]
<class 'str'>


In [188]:
def simple_split(note):
    """Dead Simple, MVP split, right down the middle of the note"""
    #These leaves a lot to be desired, but test with this MVP for now
    midpoint = int(len(note) / 2)
    return [note[:midpoint]] + [note[midpoint:]]


In [195]:
model_name = "knkarthick/MEETING_SUMMARY"
summarizer = pipeline("summarization", model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/Users/joachimpfefferkorn/repos/daily_note_organizer/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [208]:
class Section:
    def __init__(self, content):
        self.content = content
        self.tokens = tokenizer(content)
        self.num_tokens = len(self.tokens['input_ids']) #TODO there is a direct way to get these, slight hack for now

print(Section(prepared_note).num_tokens)

2718


In [263]:
import numpy as np

def split(section: Section, amount: int):
    lines = np.asarray(section.content.split("\n"), dtype=str)

    sub_arrays = np.array_split(lines, amount)

    sections = []
    #Make each sub array a string
    for array in sub_arrays:
        text = "\n".join(array)
    # then a section
    #add each section to a list
        sections.append(Section(text))
    return sections

# test_section = Section(prepared_note)
# print(test_section.num_tokens)
# splits = split(test_section, 30)
# print(len(splits))
# for i, section in enumerate(splits):
#     print(f"SECTION{i}:\n", section.content)

In [271]:
def biggest_section(sections):
    biggest_section = Section("")
    for section in sections:
        if section.num_tokens > biggest_section.num_tokens:
            biggest_section = section
    return biggest_section

output_sections = []
og_section = Section(prepared_note)
def recursive_split(input_section, split_amount):

    if input_section.num_tokens > tokenizer.model_max_length and split_amount < 99:
        split_amount += 1
        print(f"🐘 Original section must be split by {split_amount}! Section tokens: {input_section.num_tokens} Max model length: {tokenizer.model_max_length}")
        new_sections = split(og_section, split_amount)
        print(f"🪸 New sections updated with split subsections, 🔃 recursively split is recursing")
        for i, subsection in enumerate(new_sections):
            print(f"⚔️ Splitting subsection {i}")
            recursive_split(subsection, split_amount)
    else:
        print(f"🦋 Section is small enough!")
        print(f"🏄 Small enough section added to new_sections")
        output_sections.append(input_section)
        return 0



split_amount = 1
input_sections = split(Section(prepared_note), split_amount)
print(type(input_sections[0]))
# for section in sections:
#     print(section.content)

recursive_split(og_section, split_amount)
print(output_sections)


<class '__main__.Section'>
🐘 Original section must be split by 2! Section tokens: 2718 Max model length: 1024
🪸 New sections updated with split subsections, 🔃 recursively split is recursing
⚔️ Splitting subsection 0
🐘 Original section must be split by 3! Section tokens: 1394 Max model length: 1024
🪸 New sections updated with split subsections, 🔃 recursively split is recursing
⚔️ Splitting subsection 0
🦋 Section is small enough!
🏄 Small enough section added to new_sections
⚔️ Splitting subsection 1
🦋 Section is small enough!
🏄 Small enough section added to new_sections
⚔️ Splitting subsection 2
🦋 Section is small enough!
🏄 Small enough section added to new_sections
⚔️ Splitting subsection 1
🐘 Original section must be split by 3! Section tokens: 1325 Max model length: 1024
🪸 New sections updated with split subsections, 🔃 recursively split is recursing
⚔️ Splitting subsection 0
🦋 Section is small enough!
🏄 Small enough section added to new_sections
⚔️ Splitting subsection 1
🦋 Section is s

In [273]:
for i, section in enumerate(output_sections):
    print("SECTION: ", i, "\n", section.content)

SECTION:  0 #dailyNote 

spam for note testing

more preamble

# Afternoon To-Do
#meta #todoList 
## Priority
Completed: Complete Python module
Completed: Complete Bias and Variance
## Bonus/tomorrow:
Completed: Note cleanup with Aliases (see Career and Study To - Do)
To Do: Linear Algebra Note Migration
Completed: Integrate with this daily note
		To Do: Training, Test, and Dev Sets
		To Do: regularization (#merge and integrate)
		To Do: Cost and Loss
Completed: Big overhaul of Bias and Variance with regards to todays notes. 
# Machine Learning Specialization Notes:
These notes will be integrated into other notes but kept here in their entirety.
## Bias and Variance
Machine Learning Specialization Advanced Machine Learning Algorithms, Week 3

Models almost never work the first time you try them out. Let's see how we can fix them.

### Diagnosing Bias/Variance
See Bias and Variance

If you have more features, you can't visualize Bias and Variance

A more systematic way to see if you hav

In [133]:
for note in notes:
    print(type(note))
    print(len(tokenizer(note)[0]))
    #print(summarize_string(note, summarizer, tokenizer))